# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState

player = DungeonPlayerState(
    playerId=1,
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="던전을 돌면 뭐가 좋다 했지?")],    
    "dungenon_player": player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
await graph.ainvoke({
    "messages":[add_human_message("스킬 사용법이 뭐야?")]
}, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("내 아이템중에 가장 쎈 무기가 뭐야?")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: W

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 8, 'neighbors': [2, 0], 'monsters': [], 'room_id': 1, 'room_type': 'event', 'event_type': 2}, {'size': 6, 'neighbors': [1, 4], 'monsters': [{'monster_id': 0, 'pos_x': 0.07123959064483643, 'pos_y': 0.49437010288238525}, {'monster_id': 0, 'pos_x': 0.6445988416671753, 'pos_y': 0.6356164216995239}, {'monster_id': 0, 'pos_x': 0.3811793327331543, 'pos_y': 0.9721304178237915}, {'monster_id': 0, 'pos_x': 0.557853102684021, 'pos_y': 0.6665529012680054}, {'monster_id': 0, 'pos_x': 0.8378279209136963, 'pos_y': 0.44205427169799805}, {'monster_id': 0, 'pos_x': 0.8323639631271362, 'pos_y': 0.7380518913269043}, {'monster_id': 0, 'pos_x': 0.7463976144790649, 'pos_y': 0.7349321842193604}, {'monster_id': 0, 'pos_x': 0.44071531295776367, 'pos_y': 0.5027787685394287}], 'room_id': 2, 'room_type': 'monster'

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T21:10:18.307348'}, response_metadata={}, id='323f2668-548e-4f33-a55e-ff2fc7d0b624'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 로코와 상담할 수 있고, 스킬이나 스탯도 강화할 수 있단다!', additional_kwargs={'created_at': '2025-12-07T21:10:19.915123', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='d5f2b0bc-4549-43ac-9d9b-11a174126086'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:19.916461'}, response_metadata={}, id='a10bd6b3-a308-4509-a003-5bc1e7d1578a'),
 AIMessage(content='무기 스킬은 E 키, 직업 스킬은 R 키로 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-07T21:10:20.872070', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='a1e327df-e97d-41f5-a0f6-debff330e28b'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:20.873521'}, response_metadata={}, id='3b7bc9a6-db06-4beb-a0da-bcfa3af1ef41'),
 AIMessage(

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [2]:
await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?")}, config=config
)
player = DungeonPlayerState(
    playerId=1,
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="내가 잡을 수 있지?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.MONSTER_GUIDE: 'MONSTER_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 8, 'neighbors': [2, 0], 'monsters': [], 'room_id': 1, 'room_type': 'event', 'event_type': 2}, {'size': 6, 'neighbors': [1, 4], 'monsters': [{'monster_id': 0, 'pos_x': 0.07123959064483643, 'pos_y': 0.49437010288238525}, {'monster_id': 0, 'pos_x': 0.6445988416671753, 'pos_y': 0.6356164216995239}, {'monster_id': 0, 'pos_x': 0.3811793327331543, 'pos_y': 0.9721304178237915}, {'monster_id': 0, 'pos_x': 0.557853102684021, 'pos_y': 0.6665529012680054}, {'monster_id': 0, 'pos_x': 0.8378279209136963, 'pos_y': 0.44205427169799805}, {'monster_id': 0, 'pos_x': 0.8323639631271362, 'pos_y': 0.7380518913269043}, {'monster_id': 0, 'pos_x': 0.7463976144790649, 'pos_y': 0.7349321842193604}, {'monster_id': 0, 'pos_x': 0.44071531295776367, 'pos_y': 0.5027787685394287}], 'room_id': 2, 'room_type': 'mons

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T21:10:18.307348'}, response_metadata={}, id='323f2668-548e-4f33-a55e-ff2fc7d0b624'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 로코와 상담할 수 있고, 스킬이나 스탯도 강화할 수 있단다!', additional_kwargs={'created_at': '2025-12-07T21:10:19.915123', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='d5f2b0bc-4549-43ac-9d9b-11a174126086'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:19.916461'}, response_metadata={}, id='a10bd6b3-a308-4509-a003-5bc1e7d1578a'),
 AIMessage(content='무기 스킬은 E 키, 직업 스킬은 R 키로 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-07T21:10:20.872070', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='a1e327df-e97d-41f5-a0f6-debff330e28b'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:20.873521'}, response_metadata={}, id='3b7bc9a6-db06-4beb-a0da-bcfa3af1ef41'),
 AIMessage(

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [3]:
result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 8, 'neighbors': [2, 0], 'monsters': [], 'room_id': 1, 'room_type': 'event', 'event_type': 2}, {'size': 6, 'neighbors': [1, 4], 'monsters': [{'monster_id': 0, 'pos_x': 0.07123959064483643, 'pos_y': 0.49437010288238525}, {'monster_id': 0, 'pos_x': 0.6445988416671753, 'pos_y': 0.6356164216995239}, {'monster_id': 0, 'pos_x': 0.3811793327331543, 'pos_y': 0.9721304178237915}, {'monster_id': 0, 'pos_x': 0.557853102684021, 'pos_y': 0.6665529012680054}, {'monster_id': 0, 'pos_x': 0.8378279209136963, 'pos_y': 0.44205427169799805}, {'monster_id': 0, 'pos_x': 0.8323639631271362, 'pos_y': 0.7380518913269043}, {'monster_id': 0, 'pos_x': 0.7463976144790649, 'pos_y': 0.7349321842193604}, {'monster_id': 0, 'pos_x': 0.44071531295776367, 'pos_y': 0.5027787685394287}], 'room_id': 2, 'room_type': 'monster'

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T21:10:18.307348'}, response_metadata={}, id='323f2668-548e-4f33-a55e-ff2fc7d0b624'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 로코와 상담할 수 있고, 스킬이나 스탯도 강화할 수 있단다!', additional_kwargs={'created_at': '2025-12-07T21:10:19.915123', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='d5f2b0bc-4549-43ac-9d9b-11a174126086'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:19.916461'}, response_metadata={}, id='a10bd6b3-a308-4509-a003-5bc1e7d1578a'),
 AIMessage(content='무기 스킬은 E 키, 직업 스킬은 R 키로 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-07T21:10:20.872070', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='a1e327df-e97d-41f5-a0f6-debff330e28b'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:20.873521'}, response_metadata={}, id='3b7bc9a6-db06-4beb-a0da-bcfa3af1ef41'),
 AIMessage(

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [4]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="다음 방은 뭐가 나올까?") },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 8, 'neighbors': [2, 0], 'monsters': [], 'room_id': 1, 'room_type': 'event', 'event_type': 2}, {'size': 6, 'neighbors': [1, 4], 'monsters': [{'monster_id': 0, 'pos_x': 0.07123959064483643, 'pos_y': 0.49437010288238525}, {'monster_id': 0, 'pos_x': 0.6445988416671753, 'pos_y': 0.6356164216995239}, {'monster_id': 0, 'pos_x': 0.3811793327331543, 'pos_y': 0.9721304178237915}, {'monster_id': 0, 'pos_x': 0.557853102684021, 'pos_y': 0.6665529012680054}, {'monster_id': 0, 'pos_x': 0.8378279209136963, 'pos_y': 0.44205427169799805}, {'monster_id': 0, 'pos_x': 0.8323639631271362, 'pos_y': 0.7380518913269043}, {'monster_id': 0, 'pos_x': 0.7463976144790649, 'pos_y': 0.7349321842193604}, {'monster_id': 0, 'pos_x': 0.44071531295776367, 'pos_y': 0.5027787685394287}], 'room_id': 2, 'room_type

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T21:10:18.307348'}, response_metadata={}, id='323f2668-548e-4f33-a55e-ff2fc7d0b624'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 로코와 상담할 수 있고, 스킬이나 스탯도 강화할 수 있단다!', additional_kwargs={'created_at': '2025-12-07T21:10:19.915123', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='d5f2b0bc-4549-43ac-9d9b-11a174126086'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:19.916461'}, response_metadata={}, id='a10bd6b3-a308-4509-a003-5bc1e7d1578a'),
 AIMessage(content='무기 스킬은 E 키, 직업 스킬은 R 키로 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-07T21:10:20.872070', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='a1e327df-e97d-41f5-a0f6-debff330e28b'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:20.873521'}, response_metadata={}, id='3b7bc9a6-db06-4beb-a0da-bcfa3af1ef41'),
 AIMessage(

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [5]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?") },
    config=config
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 8, 'neighbors': [2, 0], 'monsters': [], 'room_id': 1, 'room_type': 'event', 'event_type': 2}, {'size': 6, 'neighbors': [1, 4], 'monsters': [{'monster_id': 0, 'pos_x': 0.07123959064483643, 'pos_y': 0.49437010288238525}, {'monster_id': 0, 'pos_x': 0.6445988416671753, 'pos_y': 0.6356164216995239}, {'monster_id': 0, 'pos_x': 0.3811793327331543, 'pos_y': 0.9721304178237915}, {'monster_id': 0, 'pos_x': 0.557853102684021, 'pos_y': 0.6665529012680054}, {'monster_id': 0, 'pos_x': 0.8378279209136963, 'pos_y': 0.44205427169799805}, {'monster_id': 0, 'pos_x': 0.8323639631271362, 'pos_y': 0.7380518913269043}, {'monster_id': 0, 'pos_x': 0.7463976144790649, 'pos_y': 0.7349321842193604}, {'monster_id': 0, 'pos_x': 0.44071531295776367, 'pos_y': 0.5027787685394287}], 'room_id': 2, 'room_type': 'monster'

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T21:10:18.307348'}, response_metadata={}, id='323f2668-548e-4f33-a55e-ff2fc7d0b624'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 로코와 상담할 수 있고, 스킬이나 스탯도 강화할 수 있단다!', additional_kwargs={'created_at': '2025-12-07T21:10:19.915123', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='d5f2b0bc-4549-43ac-9d9b-11a174126086'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:19.916461'}, response_metadata={}, id='a10bd6b3-a308-4509-a003-5bc1e7d1578a'),
 AIMessage(content='무기 스킬은 E 키, 직업 스킬은 R 키로 사용할 수 있어!', additional_kwargs={'created_at': '2025-12-07T21:10:20.872070', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='a1e327df-e97d-41f5-a0f6-debff330e28b'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T21:10:20.873521'}, response_metadata={}, id='3b7bc9a6-db06-4beb-a0da-bcfa3af1ef41'),
 AIMessage(

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [6]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
await graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는법 힌트좀 줘") },
    config=config
)
result = await graph.ainvoke(
    { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
    config=config
)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


[HumanMessage(content='히로인의 호감도 얻는법 힌트좀 줘', additional_kwargs={'created_at': '2025-12-07T21:10:28.248679'}, response_metadata={}, id='281b4bb7-86f5-4df6-a6ad-420ebba8778b'),
 AIMessage(content='로코는 따뜻한 열기와 머리 쓰다듬는 걸 좋아해요! 금속 소리나 정직함도 호감 ↑~ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 2967, 'total_tokens': 3438, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 434, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 2967, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b5ca4d2f-083c-c079-79f6-574b6a097da6', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9439b575-4fb4-449b-9ba9-3b8d3fe2b13a-0', usage_metadata={'input_tokens': 2967, 'output_tokens': 37, 'total_tokens': 3438, 'input_token_d